<a href="https://colab.research.google.com/github/Venkatarameshredd/30DaysOfWebDevelopment/blob/main/OLAP(Dw_lab3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
#load fact tables
x=pd.read_csv("/content/fact_sales.csv")
print(x)
print(x.head())

    SalesID  TimeID  LocationID  ProductID   Sales  Quantity  Profit
0         1       1           1          1  120000        80   25000
1         2       1           1          2  200000        40   45000
2         3       2           2          3   50000       120   12000
3         4       3           3          1  150000        90   30000
4         5       4           4          5  180000        30   35000
5         6       5           5          3   70000        60   15000
6         7       6           6          2  220000        45   50000
7         8       7           7          1  130000        85   28000
8         9       8           8          4   90000        25   20000
9        10       9           9          5  160000        28   32000
10       11      10           1          2  240000        50   55000
11       12      11           5          3   65000       110   14000
12       13      12           7          1  170000        95   34000
13       14      13           1   

In [4]:
#calculating total sum of sales
total_sales=x["Sales"].sum()
print(total_sales)

2115000


In [5]:
#roll_up by product-id
#Roll-up aggregates data to a higher level of abstraction by reducing detail.
roll_up_product=x.groupby("ProductID")["Sales"].sum().reset_index() #Rollup(aggregation operation)All rows having the same ProductID are placed into one group
print(roll_up_product)

   ProductID   Sales
0          1  570000
1          2  660000
2          3  185000
3          4  185000
4          5  515000


In [6]:
#Drill_Down operations(move high level data to low level data)
#break aggregated data into finer levels
drilldown = x.groupby(
    ["ProductID", "LocationID"]
)["Sales"].sum().reset_index()

print(drilldown)


    ProductID  LocationID   Sales
0           1           1  120000
1           1           3  150000
2           1           7  300000
3           2           1  440000
4           2           6  220000
5           3           2   50000
6           3           5  135000
7           4           1   95000
8           4           8   90000
9           5           4  180000
10          5           6  175000
11          5           9  160000


In [7]:
#Slice operations
#selects data by fixing one dimension to a single value, creating a sub-cube
slice_product = x[x["ProductID"] == 1] #Checks each row to see whether ProductID is equal to 1 Stores the filtered result in a new DataFrame called slice_product
print(slice_product)



SLICE: ProductID = 1
    SalesID  TimeID  LocationID  ProductID   Sales  Quantity  Profit
0         1       1           1          1  120000        80   25000
3         4       3           3          1  150000        90   30000
7         8       7           7          1  130000        85   28000
12       13      12           7          1  170000        95   34000


In [8]:
#Dice operations
#Dice selects data by applying multiple conditions on dimensions.
#Dice where ProductID = 1 AND LocationID = 1
dice = x[
    (x["ProductID"] == 1) &
    (x["LocationID"] == 1)
]

print("\nDICE: ProductID = 1 AND LocationID = 1")
print(dice)



DICE: ProductID = 1 AND LocationID = 1
   SalesID  TimeID  LocationID  ProductID   Sales  Quantity  Profit
0        1       1           1          1  120000        80   25000


In [9]:
#Pivot(Rotate Operation)(Changing Data operation)
#Pivot rotates the data cube to view data from a different perspective
pivot_table = pd.pivot_table(
    x,
    values="Sales",
    index="ProductID",
    columns="LocationID",
    aggfunc="sum",
    fill_value=0
)   #reshape and aggregate data

print("\nPIVOT TABLE: ProductID vs LocationID")
print(pivot_table)



PIVOT TABLE: ProductID vs LocationID
LocationID       1      2       3       4       5       6       7      8  \
ProductID                                                                  
1           120000      0  150000       0       0       0  300000      0   
2           440000      0       0       0       0  220000       0      0   
3                0  50000       0       0  135000       0       0      0   
4            95000      0       0       0       0       0       0  90000   
5                0      0       0  180000       0  175000       0      0   

LocationID       9  
ProductID           
1                0  
2                0  
3                0  
4                0  
5           160000  


In [10]:
#MULTIMEASURE OLAP ANALYSIS
z = x.groupby("ProductID")[["Sales", "Quantity", "Profit"]].sum().reset_index() #This code aggregates multiple business measures (sales, quantity, profit)
#at the product level using a roll-up operation.

print("\nMULTI-MEASURE ANALYSIS")
print(z)



MULTI-MEASURE ANALYSIS
   ProductID   Sales  Quantity  Profit
0          1  570000       350  117000
1          2  660000       135  150000
2          3  185000       290   41000
3          4  185000        55   41000
4          5  515000        90  103000


In [11]:
#Top 3 sales Record
top_sales = x.sort_values(by="Sales", ascending=False).head(3)

print("TOP 3 SALES RECORDS")
print(top_sales)


TOP 3 SALES RECORDS
    SalesID  TimeID  LocationID  ProductID   Sales  Quantity  Profit
10       11      10           1          2  240000        50   55000
6         7       6           6          2  220000        45   50000
1         2       1           1          2  200000        40   45000


In [12]:
#percentage contribution of each product
product_sales = x.groupby("ProductID")["Sales"].sum()
#This code calculates how much each product contributes (in percentage) to the total sales.
percentage = (product_sales / product_sales.sum()) * 100

print("PERCENTAGE CONTRIBUTION BY PRODUCT")
print(percentage)


PERCENTAGE CONTRIBUTION BY PRODUCT
ProductID
1    26.950355
2    31.205674
3     8.747045
4     8.747045
5    24.349882
Name: Sales, dtype: float64
